#### Lab 02 - Collaborative Filtering (User, Rating, Film)

In [ ]:
import pandas, warnings

warnings.filterwarnings("ignore")

#### `Rating`

In [ ]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
rating.tail()

,UserID,MovieID,Rating,Timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [ ]:
# Check Null Value on Rating Table

null = rating.isnull().sum()

null.sum()

0

In [ ]:
# Check Average Rating on Rating Table

rata_rata_rating = rating["Rating"].mean()

rata_rata_rating

3.501556983616962

In [ ]:
# Get Total User

# Our Total User is 610 User

total_user = rating["UserID"].nunique()

total_user

610

In [ ]:
# Get Total Film

# Our Total Film is 9.724 Film

total_film = rating["MovieID"].nunique()

total_film

9724

#### `Table Film`

In [ ]:
table_film = pandas.read_table("movies.csv", sep=",")

table_film.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Genre Engineering

splita = lambda val : " ".join(val.split("|"))

table_film["Genre"] = table_film["Genre"].apply(splita)

In [ ]:
# Genre Engineering

nolist = "(no genres listed)"

table_film = table_film[table_film["Genre"] != nolist]

In [ ]:
# Genre Engineering

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  table_film["Genre"] = table_film["Genre"].str.replace(i, t)

table_film.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


#### `Model Spirior`

In [ ]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate = lambda table : round((table.isnull().sum().sum() / table.size) * 100, 3)

SparseRate(rating_table)

98.3

In [ ]:
minimum_film_interaction = 10

WatchFilm = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

WatchFilm.columns = ["TotalWatch", "Rating"]

WatchFilm = WatchFilm[WatchFilm["TotalWatch"] > minimum_film_interaction]

rating = rating[rating["MovieID"].isin(WatchFilm.index)]

rating["MovieID"].nunique()

2121

In [ ]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

93.845

In [ ]:
minimum_user_interaction = 50

UserWatch = rating.groupby("UserID")[["Rating"]].count()

UserWatch = UserWatch[UserWatch["Rating"] > minimum_user_interaction]

rating = rating[rating["UserID"].isin(UserWatch.index)]

In [ ]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

90.396

#### `Main Model`

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric="cosine", algorithm="brute")

model.fit(rating_matrix)

In [ ]:
def OutputFilm(title, limit=5):

  movies_index = table_film[table_film["Title"] == title]
  movies_index = movies_index["MovieID"].values[0]
  movies_index = rating_reset[rating_reset["MovieID"] == movies_index].index[0]

  limit += 1

  distances, indices = model.kneighbors(rating_matrix[movies_index], n_neighbors=limit)
  sorter = list(zip(indices.flatten().tolist(), distances.flatten().tolist()))
  indices = sorted(sorter, key=lambda x: x[1])

  molist = [i[0] for i in indices]
  molist = [i for i in rating_reset.iloc[molist, 0].values][1:]
  result = table_film[table_film["MovieID"].isin(molist)]

  return result

In [ ]:
title = "Avengers, The (2012)"

OutputFilm(title, 15)

In [ ]:
title = table_film[table_film["MovieID"] == 2]["Title"].values[0]

OutputFilm(title, 10)

In [ ]:
title = "Forrest Gump (1994)"

OutputFilm(title)

#### `First Time User on Rating and Genre`

In [ ]:
optional = ["Action", "Children", "Fantasy", "Romance"]